# ECE 590 - Lab2 - LeNet5 Cifar 10 - Anthony DiSpirito

## My Info
* **Name: Anthony DiSpirito**
* **NetID: ad424**
* **Class: ECE 590 - Comp Eng ML and DL**
* **Title: Lab2 - My Model**
* **Date: 10/2/19**

## Build My Model

### Import Statements

In [1]:
import argparse
import os, sys
import math
import random
import time
import datetime
import numpy as py

# Import pytorch dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.backends.cudnn as cudnn
from tqdm import tqdm_notebook as tqdm

# You cannot change this line.
from tools.dataloader import CIFAR10

### My Model Class

In [2]:
""" 
Assignment 2(a)
Build the LeNet-5 model by following table 1 or figure 1.

You can also insert batch normalization and leave the LeNet-5 
with batch normalization here for assignment 3(c).
"""
# Create the neural network module:
class convBlock(nn.Module):
    def __init__(self, k, channel_in, channel_out, layers, padding):
        super(convBlock, self).__init__()
        self.layers = layers
        if padding == False:
            self.conv1 = nn.Conv2d(channel_in, channel_out, k, padding=0, dilation=1,
                                   groups=1, bias=True, padding_mode='zeros')
            self.bn_conv1 = nn.BatchNorm2d(channel_out, eps=1e-05,momentum=0.1, 
                                       affine=True, track_running_stats=True)
            if layers > 1:
                self.conv2 = nn.Conv2d(channel_out, channel_out, k, padding=0, dilation=1,
                                   groups=1, bias=True, padding_mode='zeros')
                self.bn_conv2 = nn.BatchNorm2d(channel_out, eps=1e-05,momentum=0.1, 
                                       affine=True, track_running_stats=True)
                if layers > 2:
                    self.conv3 = nn.Conv2d(channel_out, channel_out, k, padding=0, dilation=1,
                                   groups=1, bias=True, padding_mode='zeros')
                    self.bn_conv3 = nn.BatchNorm2d(channel_out, eps=1e-05,momentum=0.1, 
                                       affine=True, track_running_stats=True)
                    if layers > 3:
                        self.conv4 = nn.Conv2d(channel_out, channel_out, k, padding=0, dilation=1,
                                   groups=1, bias=True, padding_mode='zeros')
                        self.bn_conv4 = nn.BatchNorm2d(channel_out, eps=1e-05,momentum=0.1, 
                                       affine=True, track_running_stats=True)
            
        else:
            self.conv1 = nn.Conv2d(channel_in, channel_out, k, padding=k//2, dilation=1,
                                   groups=1, bias=True, padding_mode='zeros')
            self.bn_conv1 = nn.BatchNorm2d(channel_out, eps=1e-05,momentum=0.1, 
                                       affine=True, track_running_stats=True)
            if layers > 1:
                self.conv2 = nn.Conv2d(channel_out, channel_out, k, padding=k//2, dilation=1,
                                   groups=1, bias=True, padding_mode='zeros')
                self.bn_conv2 = nn.BatchNorm2d(channel_out, eps=1e-05,momentum=0.1, 
                                       affine=True, track_running_stats=True)
                if layers > 2:
                    self.conv3 = nn.Conv2d(channel_out, channel_out, k, padding=k//2, dilation=1,
                                   groups=1, bias=True, padding_mode='zeros')
                    self.bn_conv3 = nn.BatchNorm2d(channel_out, eps=1e-05,momentum=0.1, 
                                       affine=True, track_running_stats=True)
                    if layers > 3:
                        self.conv4 = nn.Conv2d(channel_out, channel_out, k, padding=k//2, dilation=1,
                                   groups=1, bias=True, padding_mode='zeros')
                        self.bn_conv4 = nn.BatchNorm2d(channel_out, eps=1e-05,momentum=0.1, 
                                       affine=True, track_running_stats=True)
    def forward(self, x):
        out = F.elu(self.bn_conv1(self.conv1(x)))
        if self.layers > 1:
            out = F.elu(self.bn_conv2(self.conv2(out)))
            if self.layers > 2:
                out = F.elu(self.bn_conv3(self.conv3(out)))
                if self.layers > 3:
                    out = F.elu(self.bn_conv4(self.conv4(out)))
        out = F.max_pool2d(out, 2)
        return out
        
class TONY_NET(nn.Module):
    def __init__(self):
        super(TONY_NET, self).__init__()
        # padding = k//2
        #self.Dropout2d(p=0.5, inplace=False)
        self.convBlock1 = convBlock(k = 3, channel_in = 3, channel_out = 64, layers = 2, padding = True)
        self.convBlock2 = convBlock(k = 3, channel_in = 64, channel_out = 128, layers = 2, padding = True)
        self.convBlock3 = convBlock(k = 3, channel_in = 128, channel_out = 256, layers = 4, padding = True)
        self.convBlock4 = convBlock(k = 3, channel_in = 256, channel_out = 512, layers = 4, padding = True)
        self.convBlock5 = convBlock(k = 3, channel_in = 512, channel_out = 512, layers = 4, padding = True)
        orig_input_size = 32 # 32x32
        number_of_conv_block_poolings = 5
        pooling_size = 2 # 2x2 with stride 2
        num_features = int(512*(orig_input_size*(1/(pooling_size**number_of_conv_block_poolings)))**2)
        self.fc1 = nn.Linear(num_features, num_features, bias = True)
        torch.nn.init.xavier_normal_(self.fc1.weight)
        self.dropout_fc1 = nn.Dropout(p = 0.5)
        self.fc2 = nn.Linear(num_features, num_features//4, bias = True)
        torch.nn.init.xavier_normal_(self.fc2.weight)
        self.dropout_fc2 = nn.Dropout(p = 0.5)
        #------------------
        self.fc3 = nn.Linear(num_features//4, num_features//8, bias = True)
        torch.nn.init.xavier_normal_(self.fc3.weight)
        self.dropout_fc3 = nn.Dropout(p = 0.5)
        #------------------
        self.fc4 = nn.Linear(num_features//8, 10, bias = True)
        torch.nn.init.xavier_normal_(self.fc4.weight)
        return
    def forward(self, x):
        out = self.convBlock1(x)
        out = self.convBlock2(out)
        out = self.convBlock3(out)
        out = self.convBlock4(out)
        out = self.convBlock5(out)
        out = out.view(out.size(0), -1)
        out = F.elu(self.fc1(out))
        out = self.dropout_fc1(out)
        out = F.elu(self.fc2(out))
        out = self.dropout_fc2(out)   
        out = F.elu(self.fc3(out))
        out = self.dropout_fc3(out)
        out = F.elu(self.fc4(out))
        return out

### Setting Hyperparameters

In [3]:
"""
Hyperparameter optimization in assignment 4(a), 4(b) can be 
conducted here.
Be sure to leave only your best hyperparameter combination
here and comment the original hyperparameter settings.
"""

# Setting some hyperparameters
TRAIN_BATCH_SIZE = 256
VAL_BATCH_SIZE = 200
INITIAL_LR = 0.0078
MOMENTUM = 0.9
REG = 5e-4
EPOCHS = 400
DATAROOT = "./data"
CHECKPOINT_PATH = "./saved_model"

In [4]:
"""
Assignment 2(b)
Write functions to load dataset and preprocess the incoming data. 
We recommend that the preprocess scheme \textbf{must} include 
normalize, standardization, batch shuffling to make sure the training 
process goes smoothly. 
The preprocess scheme may also contain some data augmentation methods 
(e.g., random crop, random flip, etc.). 

Reference value for mean/std:

mean(RGB-format): (0.4914, 0.4822, 0.4465)
std(RGB-format): (0.2023, 0.1994, 0.2010)


NOTE: Considering this process has strong corrlelation with assignment 3(b), 
please leave the data preprocessing method which can achieve the highest 
validation accuracy here. You can include your original data augmentation
method as comments and denotes the accuracy difference between these two 
methods.
"""
# Specify preprocessing function.
# Reference mean/std value
mean_RGB = (0.4914, 0.4822, 0.4465)
std_RGB = (0.2023, 0.1994, 0.2010)
input_img_size = (32,32)
percent_crop = 0.80 # 80 percent left after crop
data_augment_list = [#transforms.RandomResizedCrop(size = input_img_size, scale=(percent_crop, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2),
                     transforms.RandomVerticalFlip(p = 0.20),
                     transforms.RandomHorizontalFlip(p = 0.40),
                     transforms.RandomApply([transforms.RandomCrop(size = input_img_size, padding=4, pad_if_needed=False, fill=0, padding_mode='constant')], p = 0.25),
                     transforms.RandomApply([transforms.RandomAffine(degrees = 45, translate=(0.15,0.15), scale=None, shear=None, resample=False, fillcolor=0)], p = 0.25),
                     transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05)], p = 0.20)]
transform_train = transforms.Compose([transforms.Resize(input_img_size),
                                      transforms.RandomOrder(data_augment_list),
                                      transforms.ToTensor(),
                                      transforms.RandomErasing(p=0.20, scale=(0.01, 0.3), ratio=(0.3, 3.3), value=0, inplace=False),
                                      transforms.Normalize(mean_RGB, std_RGB)])
transform_val = transforms.Compose([transforms.Resize(input_img_size),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean_RGB, std_RGB)])

In [5]:
# Call the dataset Loader
num_threads = 16
trainset = CIFAR10(root=DATAROOT, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=num_threads//2)
valset = CIFAR10(root=DATAROOT, train=False, download=True, transform=transform_val)
valloader = torch.utils.data.DataLoader(valset, batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=num_threads//2)

Using downloaded and verified file: ./data\cifar10_trainval.tar.gz
Extracting ./data\cifar10_trainval.tar.gz to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data\cifar10_trainval.tar.gz
Extracting ./data\cifar10_trainval.tar.gz to ./data
Files already downloaded and verified


In [6]:
# Specify the device for computation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = TONY_NET()
net = net.to(device)
if device =='cuda':
    print("Train on GPU...")
else:
    print("Train on CPU...")

Train on GPU...


In [7]:
# FLAG for loading the pretrained model
TRAIN_FROM_SCRATCH = True
# Code for loading checkpoint and recover epoch id.
CKPT_PATH = "./saved_model/model.h5"
def get_checkpoint(ckpt_path):
    try:
        ckpt = torch.load(ckpt_path)
    except Exception as e:
        print(e)
        return None
    return ckpt

ckpt = get_checkpoint(CKPT_PATH)
if ckpt is None or TRAIN_FROM_SCRATCH:
    if not TRAIN_FROM_SCRATCH:
        print("Checkpoint not found.")
    print("Training from scratch ...")
    start_epoch = 0
    current_learning_rate = INITIAL_LR
else:
    print("Successfully loaded checkpoint: %s" %CKPT_PATH)
    net.load_state_dict(ckpt['net'])
    start_epoch = ckpt['epoch'] + 1
    current_learning_rate = ckpt['lr']
    print("Starting from epoch %d " %start_epoch)

print("Starting from learning rate %f:" %current_learning_rate)

Training from scratch ...
Starting from learning rate 0.007800:


In [8]:
"""
Assignment 2(c)
In the targeted classification task, we use cross entropy loss with L2 
regularization as the learning object.
You need to formulate the cross-entropy loss function in PyTorch.
You should also specify a PyTorch Optimizer to optimize this loss function.
We recommend you to use the SGD-momentum with an initial learning rate 0.01 
and momentum 0.9 as a start.
"""
# Create loss function and specify regularization
criterion = nn.CrossEntropyLoss()
# Add optimizer
optimizer = optim.SGD(net.parameters(), INITIAL_LR, MOMENTUM, weight_decay=REG, nesterov = True)

In [9]:
"""
Assignment 3(a)
Start the training process over the whole CIFAR-10 training dataset. 
For sanity check, you are required to report the initial loss value at 
the beginning of the training process and briefly justify this value. 
Run the training process for \textbf{a maximum of 30} epochs and you 
should be able to reach around \textbf{65\%} accuracy on the validation 
dataset.
"""
# Start the training/validation process
# The process should take about 5 minutes on a GTX 1070-Ti
# if the code is written efficiently.
global_step = 0
best_val_acc = 0

for i in range(start_epoch, EPOCHS):
    print(datetime.datetime.now())
    # Switch to train mode
    net.train()
    print("Epoch %d:" %i)

    total_examples = 0
    correct_examples = 0

    train_loss = 0
    train_acc = 0
    # Train the training dataset for 1 epoch.
    print(len(trainloader))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # Copy inputs to device
        inputs = inputs.to(device)
        targets = targets.to(device)
        # Zero the gradient
        optimizer.zero_grad()
        # Generate output
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        # Now backward loss
        loss.backward()
        # Apply gradient
        optimizer.step()
        # Calculate predicted labels
        _, predicted = torch.max(outputs.data, 1)
        # Calculate accuracy
        total_examples += targets.size(0)
        correct_examples += predicted.eq(targets.data).sum().item()
        train_loss += loss.data.item()

        global_step += 1
        if global_step % 100 == 0:
            avg_loss = train_loss / (batch_idx + 1)
        pass
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f %%" %(avg_loss, avg_acc*100))
    print(datetime.datetime.now())
    # Validate on the validation dataset
    print("Validation...")
    total_examples = 0
    correct_examples = 0
    
    net.eval()

    val_loss = 0
    val_acc = 0
    # Disable gradient during validation
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valloader):
            # Copy inputs to device
            inputs = inputs.to(device)
            targets = targets.to(device)
            # Zero the gradient
            optimizer.zero_grad()
            # Generate output from the DNN.
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            # Calculate predicted labels
            _, predicted = torch.max(outputs.data, 1)
            # Calculate accuracy
            total_examples += targets.size(0)
            correct_examples += predicted.eq(targets.data).sum().item()
            val_loss += loss.data.item()

    avg_loss = val_loss / len(valloader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f %%" %(avg_loss, avg_acc*100))

        
    """
    Assignment 4(b)
    Learning rate is an important hyperparameter to tune. Specify a 
    learning rate decay policy and apply it in your training process. 
    Briefly describe its impact on the learning curveduring your 
    training process.    
    Reference learning rate schedule: 
    decay 0.98 for every 2 epochs. You may tune this parameter but 
    minimal gain will be achieved.
    Assignment 4(c)
    As we can see from above, hyperparameter optimization is critical 
    to obtain a good performance of DNN models. Try to fine-tune the 
    model to over 70% accuracy. You may also increase the number of 
    epochs to up to 100 during the process. Briefly describe what you 
    have tried to improve the performance of the LeNet-5 model.
    """
    DECAY_EPOCHS = 2
    DECAY = 0.96#0.98
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = INITIAL_LR*(DECAY**(i//DECAY_EPOCHS))   
        for param_group in optimizer.param_groups:
            # Assign the learning rate parameter
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %(current_learning_rate*(DECAY**(i//DECAY_EPOCHS))))
    
    # Save for checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_PATH):
            os.makedirs(CHECKPOINT_PATH)
        print("Saving ...")
        state = {'net': net.state_dict(),
                 'epoch': i,
                 'lr': current_learning_rate}
        torch.save(state, os.path.join(CHECKPOINT_PATH, 'model.h5'))

print("Optimization finished.")

2019-10-02 12:51:29.703669
Epoch 0:
176
Training loss: 2.3897, Training accuracy: 11.6733 %
2019-10-02 12:51:51.224654
Validation...
Validation loss: 2.1930, Validation accuracy: 10.0000 %
Saving ...
2019-10-02 12:52:02.615532
Epoch 1:
176
Training loss: 2.1923, Training accuracy: 16.3467 %
2019-10-02 12:52:23.266919
Validation...
Validation loss: 1.9514, Validation accuracy: 22.0600 %
Saving ...
2019-10-02 12:52:33.113622
Epoch 2:
176
Training loss: 2.0179, Training accuracy: 24.5600 %
2019-10-02 12:52:53.717547
Validation...
Validation loss: 1.8083, Validation accuracy: 34.1000 %
Saving ...
2019-10-02 12:53:01.602251
Epoch 3:
176
Training loss: 1.7823, Training accuracy: 34.4689 %
2019-10-02 12:53:22.193385
Validation...
Validation loss: 1.5385, Validation accuracy: 44.9000 %
Current learning rate has decayed to 0.007645
Saving ...
2019-10-02 12:53:33.336563
Epoch 4:
176
Training loss: 1.5686, Training accuracy: 44.8244 %
2019-10-02 12:53:53.921013
Validation...
Validation loss: 1.23